In [1]:
import pandas as pd
import numpy as np
import os
import json
import pyodbc
import csv
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from etl.load_data import create_access_token
from etl.load_data import get_auction_data

## Retrieve most recent data

In [2]:
def get_data(sql, config):
    conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + config['db_host'] +';DATABASE=' + config['db_name'] +';UID='+config['db_user'] +';PWD='+ config['db_password'] )
    cursor = conn.cursor()

    cursor.execute(sql)

    result = cursor.fetchall()

    headers = [column[0] for column in cursor.description]
    
    cursor.close()
    conn.close()
    
    return result, headers

In [3]:
with open('etl/config.json') as json_data:
    config = json.load(json_data)

with open("sql/get_data.sql", "r") as f:
    sql = f.read()

In [ ]:
result, headers = get_data(sql, config)

In [ ]:
with open('auction_data.csv', 'w', newline='') as out:
    csv_out = csv.writer(out)
    csv_out.writerow(headers)
    for row in result:
        csv_out.writerow(row)

## Read data from CSV

In [ ]:
df = pd.read_csv('auction_data.csv')
df.head(5)

In [ ]:
df.shape

In [ ]:
df['TimeLeft'].value_counts()

### Since we can't know if an item was sold or just expired, we'll focus on VERY_LONG time left items (between 12 and 48 hours left)

In [ ]:
df = df[df['TimeLeft'] == 'VERY_LONG']

In [ ]:
df.loc[:,'Sold'] = df['TimesSeen'] <= 12

In [ ]:
columns = ['Id', 'ItemName', 'Quantity', 'TimeLeft', 'ItemClass', 'TimesSeen', 'Sold']
df[columns].head(5)

# Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
columns = ['BidGold', 'BuyoutGold', 'Quantity', 'TimesSeen', 'IsEquippable']
df[columns].describe()

## Percentage of items sold

In [ ]:
df_count = df['Sold'].value_counts(normalize=True)
df_count = df_count.mul(100)
df_count = df_count.rename('Percent').reset_index()
df_count['Sold'] = df_count['index'] 

g = sns.catplot(x='Sold', y='Percent', kind='bar', data=df_count)
for p in g.ax.patches:
    txt = str(p.get_height().round(2)) + '%'
    txt_x = p.get_x()
    txt_y = p.get_height() + 3
    g.ax.text(txt_x,txt_y,txt)

## Most popular Item

In [ ]:
items_sold = df[df['Sold'] == 1]

count_items = items_sold.groupby(['ItemName']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)

print(count_items.head(10))

plot = sns.barplot(x='ItemName', y='Counts', data=count_items[:10])
for item in plot.get_xticklabels():
    item.set_rotation(45)

## Most popular Item Class

In [ ]:
items_sold = df[df['Sold'] == 1]

count_items = items_sold.groupby(['ItemClass']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)

print(count_items.head(100))

plot = sns.barplot(x='ItemClass', y='Counts', data=count_items[:5])
for item in plot.get_xticklabels():
    item.set_rotation(45)

## Most popular Item Subclass

In [ ]:
items_sold = df[df['Sold'] == 1]

count_items = items_sold.groupby(['ItemSubClass']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)

print(count_items.head(10))

plot = sns.barplot(x='ItemSubClass', y='Counts', data=count_items[:10])
for item in plot.get_xticklabels():
    item.set_rotation(45)

## Most sold quality tipe

In [ ]:
items_sold = df[df['Sold'] == 1]

count_items = items_sold.groupby(['Quality']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)

print(count_items.head(10))

plot = sns.barplot(x='Quality', y='Counts', data=count_items[:10])
for item in plot.get_xticklabels():
    item.set_rotation(45)

## Are most sold items equippable?

In [ ]:
items_sold = df[df['Sold'] == 1]

count_items = items_sold.groupby(['IsEquippable']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)[:10]

print(count_items.head())

plot = sns.barplot(x='IsEquippable', y='Counts', data=count_items)

## Best week day to sell

In [ ]:
import datetime

df['WeekDay'] = df.apply(lambda row: datetime.datetime(row['FirstTimeSeenYear'], row['FirstTimeSeenMonth'], row['FirstTimeSeenDay']).weekday(), axis=1)

count_items = df.groupby(['WeekDay']).size().reset_index(name='Counts')
count_items = count_items.sort_values(by=['Counts'], ascending=False)[:10]

print(count_items.head(10))

plot = sns.barplot(x='WeekDay', y='Counts', data=count_items)

# Feature Engineering

Unit price

In [ ]:
df['UnitPrice'] = df['BuyoutGold'] / df['Quantity']
df[['Id', 'ItemId', 'BuyoutGold', 'Quantity', 'UnitPrice']].head()

Historical sold price

In [ ]:
historical_price = df[df['Sold'] == True].groupby(by=['ItemId'])['UnitPrice'].median().reset_index(name='HistoricalPrice')

df_merged = pd.merge(df, historical_price, on=['ItemId'], how='left')
df_merged['HistoricalPrice'] = df_merged['HistoricalPrice'].fillna(0)
df_merged[['ItemId', 'UnitPrice', 'HistoricalPrice']].head(5)

Add the median competitor price to the dataset, this should be used to predict a buyout price according to the current state of market

In [ ]:
median_competitor_price = df.groupby(by=['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour'])['UnitPrice'].median().reset_index(name='MedianCompetitorPrice')

df_merged = pd.merge(df_merged, median_competitor_price, on=['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour'], how='left')
df_merged['MedianCompetitorPrice'] = df_merged['MedianCompetitorPrice'].fillna(0)
df_merged[['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour', 'UnitPrice', 'MedianCompetitorPrice']].head(3)

Add the lowest price for that item

In [ ]:
lowest_competitor_price = df[df['UnitPrice'] > 0].groupby(by=['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour'])['UnitPrice'].min().reset_index(name='LowestCompetitorPrice')

df_merged = pd.merge(df_merged, lowest_competitor_price, on=['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour'], how='left')
df_merged['LowestCompetitorPrice'] = df_merged['LowestCompetitorPrice'].fillna(0)
df_merged[['ItemId', 'FirstTimeSeenYear', 'FirstTimeSeenMonth', 'FirstTimeSeenDay', 'FirstTimeSeenHour', 'UnitPrice', 'LowestCompetitorPrice']].head(3)

# Data Preparation

In [ ]:
df_sold = df_merged[df_merged['Sold'] == True]
df_sold[['ItemId', 'Quantity', 'HistoricalPrice', 'MedianCompetitorPrice', 'LowestCompetitorPrice', 'UnitPrice']].head(10)

In [ ]:
numerical_columns = ['Quantity', 'HistoricalPrice', 'MedianCompetitorPrice', 'LowestCompetitorPrice']

categorical_columns = []

In [ ]:
X = df_sold[numerical_columns + categorical_columns]
y = df_sold['UnitPrice']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

column_transformer = make_column_transformer(
    (OneHotEncoder(sparse=False, handle_unknown='ignore'), categorical_columns),
    remainder='passthrough'
)

X_train = column_transformer.fit_transform(X)
y_train = np.array(y)

# Model Selection

In [ ]:
m = 50000

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

cross_val_score(reg, X_train[:m], y_train[:m], cv=3, scoring='neg_mean_absolute_error').mean()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()

cross_val_score(reg, X_train[:m], y_train[:m], scoring='neg_mean_absolute_error').mean()

In [ ]:
from xgboost import XGBRegressor

reg = XGBRegressor()

cross_val_score(reg, X_train[:m], y_train[:m], scoring='neg_mean_absolute_error').mean()

### Full data

In [ ]:
cross_val_score(reg, X_train, y_train, scoring='neg_mean_absolute_error').mean()

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
features = np.array(column_transformer.transformers_[0][1].get_feature_names(categorical_columns).tolist() + numerical_columns)

In [ ]:
sorted_idx = reg.feature_importances_.argsort()

plt.figure(figsize=(8,12))
plt.barh(features[sorted_idx], reg.feature_importances_[sorted_idx])

# Finetuning

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = [
    {'randomforestregressor__n_estimators': [3, 10, 30], 'randomforestregressor__max_features': [2, 4, 6, 8]},
    {'randomforestregressor__bootstrap': [False], 'randomforestregressor__n_estimators': [3, 10], 'randomforestregressor__max_features': [2, 3, 4]}
]

reg = RandomForestRegressor()

pipeline = make_pipeline(
    column_transformer,
    reg
)


grid_search = GridSearchCV(pipeline, param_grid, cv=5,scoring='neg_mean_absolute_error',return_train_score=True)

grid_search.fit(X, y)

In [ ]:
grid_search.best_params_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
X.columns

In [ ]:
sorted_idx = grid_search.best_estimator_._final_estimator.feature_importances_.argsort()

plt.figure(figsize=(8,24))
plt.barh(X.columns[sorted_idx], grid_search.best_estimator_._final_estimator.feature_importances_[sorted_idx])

# Save model

In [ ]:
import pickle

filename = 'model.sav'
pickle.dump(reg, open(filename, 'wb'))

# Load model

In [4]:
import pickle

filename = 'model.sav'
reg = pickle.load(open(filename, 'rb'))

# Data for prediction

Historical data from database

In [5]:
result, headers = get_data(sql, config)

df_historical = pd.DataFrame.from_records(result, columns=headers)
df_historical = df_historical[df_historical['BuyoutGold'] > 0]
df_historical = df_historical[df_historical['TimeLeft'] == 'VERY_LONG']
df_historical.loc[:,'Sold'] = df_historical['TimesSeen'] <= 12
df_historical['UnitPrice'] = df_historical['BuyoutGold'] / df_historical['Quantity']
df_historical = df_historical[df_historical['Sold'] == True]

df_historical.head()

,Id,ItemId,BidGold,BuyoutGold,Quantity,TimeLeft,Rand,Seed,FirstTimeSeenYear,FirstTimeSeenMonth,...,ItemSubClass,PurchasePriceGold,PurchasePriceSilver,SellPriceGold,SellPriceSilver,MaxCount,IsEquippable,IsStackable,Sold,UnitPrice
5040,1420447020,22797,74.090000,78.000000,20,VERY_LONG,NaN,NaN,2021,9,...,Herb,0,50,0,12,0,False,True,True,3.900000
5056,1420450045,1988,2.450000,2.970000,1,VERY_LONG,NaN,NaN,2021,9,...,Mail,1,42,0,28,0,True,False,True,2.970000
5061,1420450285,3185,0.800000,1.200000,1,VERY_LONG,2002.0,1.507757e+09,2021,9,...,Staff,4,4,0,80,0,True,False,True,1.200000
5062,1420451431,32067,75.250000,76.680000,10,VERY_LONG,NaN,NaN,2021,9,...,Elixir,0,60,0,15,0,False,True,True,7.668000
5063,1420451456,32067,75.250000,76.680000,10,VERY_LONG,NaN,NaN,2021,9,...,Elixir,0,60,0,15,0,False,True,True,7.668000


Item data

In [6]:
with open("sql/get_items.sql", "r") as f:
    sql_items = f.read()
    
result, headers = get_data(sql_items, config)

items = pd.DataFrame.from_records(result, columns=headers)
items['ItemId'] = items['Id']
items['SellPrice'] = items['SellPriceGold'] + items['SellPriceSilver'] / 100.0

items.head(3)

,Id,Name,Quality,Level,RequiredLevel,ItemClass,ItemSubClass,PurchasePriceGold,PurchasePriceSilver,SellPriceGold,SellPriceSilver,MaxCount,IsEquippable,IsStackable,ItemId,SellPrice
0,38,Recruit's Shirt,Common,1,0,Armor,Miscellaneous,0,0,0,0,0,True,False,38,0.0
1,45,Squire's Shirt,Common,1,0,Armor,Miscellaneous,0,0,0,0,0,True,False,45,0.0
2,51,Neophyte's Boots,Common,1,0,Armor,Miscellaneous,0,0,0,0,0,True,False,51,0.0


Get actual auction data

In [7]:
auctions = get_auction_data(save=False)

df_actual = pd.DataFrame.from_records(auctions, columns=['Id', 'ItemId', 'BidGold', 'BidSilver', 'BuyoutGold', 'BuyoutSilver', 'Quantity', 'TimeLeft', 'Rand', 'Seed'])
df_actual['BuyoutGold'] = df_actual['BuyoutGold'] + (df_actual['BuyoutSilver'] / 100.0)
df_actual['UnitPrice'] = df_actual['BuyoutGold'] / df_actual['Quantity']
df_actual = df_actual[df_actual['BuyoutGold'] > 0]

df_actual.head()

Token created
Request done
69351 auctions processed.


,Id,ItemId,BidGold,BidSilver,BuyoutGold,BuyoutSilver,Quantity,TimeLeft,Rand,Seed,UnitPrice
0,1447690364,14182,3,0,4.00,0,1,SHORT,768,579361792,4.00
1,1447691692,15334,3,39,4.77,77,1,SHORT,764,2070439680,4.77
2,1447691858,5116,1,4,1.60,60,1,SHORT,None,None,1.60
3,1447691860,5116,1,4,1.60,60,1,SHORT,None,None,1.60
4,1447691864,5116,1,4,1.60,60,1,SHORT,None,None,1.60


In [8]:
historical_price = df_historical.groupby(by=['ItemId'])['UnitPrice'].median().reset_index(name='HistoricalPrice')
median_competitor_price = df_actual.groupby(by=['ItemId'])['UnitPrice'].median().reset_index(name='MedianCompetitorPrice')
lowest_competitor_price = df_actual[df_actual['UnitPrice'] > 0].groupby(by=['ItemId'])['UnitPrice'].min().reset_index(name='LowestCompetitorPrice')

# Model Prediction

In [9]:
predict = pd.read_csv('predict.csv')
predict.head()

,ItemId,Quantity
0,23424,1
1,23425,1
2,22573,1
3,24819,1
4,23427,1


In [10]:
df_merged = pd.merge(predict, historical_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, median_competitor_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, lowest_competitor_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, items[['ItemId', 'Name', 'Quality', 'ItemClass']], on=['ItemId'], how='left')

df_merged['HistoricalPrice'] = df_merged['HistoricalPrice'].fillna(0)
df_merged['MedianCompetitorPrice'] = df_merged['MedianCompetitorPrice'].fillna(0)
df_merged['LowestCompetitorPrice'] = df_merged['LowestCompetitorPrice'].fillna(0)

df_merged.head(10)

,ItemId,Quantity,HistoricalPrice,MedianCompetitorPrice,LowestCompetitorPrice,Name,Quality,ItemClass
0,23424,1,0.6610,0.8095,0.530000,Fel Iron Ore,Common,Trade Goods
1,23425,1,1.2395,1.2500,1.137500,Adamantite Ore,Common,Trade Goods
2,22573,1,0.2990,0.2290,0.188333,Mote of Earth,Common,Trade Goods
3,24819,1,17.2400,0.0000,0.000000,Felstone Leggings,Uncommon,Armor
4,23427,1,0.6300,0.5500,0.270000,Eternium Ore,Uncommon,Trade Goods
5,23436,1,79.5950,82.4800,77.000000,Living Ruby,Rare,Gem


In [12]:
quantity = 1
predictions = reg.predict(df_merged[['Quantity', 'HistoricalPrice', 'MedianCompetitorPrice', 'LowestCompetitorPrice']]) * quantity
 
df_merged['RecommendedPrice'] = predictions
df_merged.to_csv('predictions.csv')

# Get best offers

In [ ]:
df_merged = pd.merge(df_actual, historical_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, median_competitor_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, lowest_competitor_price, on=['ItemId'], how='left')
df_merged = pd.merge(df_merged, items[['ItemId', 'Name', 'Quality', 'ItemClass']], on=['ItemId'], how='left')

df_merged['HistoricalPrice'] = df_merged['HistoricalPrice'].fillna(0)
df_merged['MedianCompetitorPrice'] = df_merged['MedianCompetitorPrice'].fillna(0)
df_merged['LowestCompetitorPrice'] = df_merged['LowestCompetitorPrice'].fillna(0)

df_merged.head()

In [ ]:
predictions = reg.predict(df_merged[['Quantity', 'HistoricalPrice', 'MedianCompetitorPrice', 'LowestCompetitorPrice']])

df_predictions = pd.DataFrame(predictions, columns=['RecommendedPrice'])
df_predictions['Id'] = df_merged[['Id']]

df_predictions.head()

In [ ]:
interest_columns = ['Id', 'ItemId', 'Name', 'Quantity', 'HistoricalPrice', 'MedianCompetitorPrice', 'LowestCompetitorPrice', 'UnitPrice', 'BuyoutGold']

df_merged_recommended = pd.merge(df_merged[interest_columns], df_predictions, on=['Id'], how='left')
df_merged_recommended = df_merged_recommended[df_merged_recommended['HistoricalPrice'] > 0]

df_merged_recommended['PriceDifference'] = df_merged_recommended['RecommendedPrice'] - df_merged_recommended['BuyoutGold']
df_merged_recommended['RelativeCompetitorPrice'] = df_merged_recommended['LowestCompetitorPrice'] / df_merged_recommended['MedianCompetitorPrice']
df_merged_recommended['RelativeHistoricalPrice'] = df_merged_recommended['LowestCompetitorPrice'] / df_merged_recommended['HistoricalPrice']

df_merged_recommended[df_merged_recommended['PriceDifference'] >= 1].to_csv('recommended_sales.csv')
df_merged_recommended[df_merged_recommended['PriceDifference'] >= 1].head()

# Get items sold in auction at a price lower than vendor price

In [ ]:
columns = ['Name', 'UnitPrice', 'SellPrice', 'Underpriced']

df_merged = pd.merge(df_actual, items, on=['ItemId'], how='left')
df_merged['Underpriced'] = df_merged['UnitPrice'] < df_merged['SellPrice']
df_merged[columns].head()

In [ ]:
df_merged[df_merged['Underpriced'] == True][columns].to_csv('underpriced.csv')